## ***Step 1:*** Download Transformers Library

In [1]:
%%capture
!pip install transformers

In [2]:
import torch
from transformers import AutoTokenizer,BertTokenizerFast

## ***Step 2:*** Load the fine tuned model

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Define the bert tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Load the fine-tuned modeol
model = torch.load("/content/drive/MyDrive/finetunedmodelHotpot",map_location=torch.device('cpu'))

model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

## ***Step 3:*** Make the prediction and evaluate it

Here I used some useful functions from the evaluation script of SQuAD dataset 2.0 so as to evaluate my fine-tuned model.

In [5]:
def predict(context,query):

  inputs = tokenizer.encode_plus(query, context, return_tensors='pt')

  outputs = model(**inputs)
  answer_start = torch.argmax(outputs[0])  # get the most likely beginning of answer with the argmax of the score
  answer_end = torch.argmax(outputs[1]) + 1

  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))

  return answer

def normalize_text(s):
  """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
  import string, re

  def remove_articles(text):
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    return re.sub(regex, " ", text)

  def white_space_fix(text):
    return " ".join(text.split())

  def remove_punc(text):
    exclude = set(string.punctuation)
    return "".join(ch for ch in text if ch not in exclude)

  def lower(text):
    return text.lower()

  return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
  pred_tokens = normalize_text(prediction).split()
  truth_tokens = normalize_text(truth).split()

  # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
  if len(pred_tokens) == 0 or len(truth_tokens) == 0:
    return int(pred_tokens == truth_tokens)

  common_tokens = set(pred_tokens) & set(truth_tokens)

  # if there are no common tokens then f1 = 0
  if len(common_tokens) == 0:
    return 0

  prec = len(common_tokens) / len(pred_tokens)
  rec = len(common_tokens) / len(truth_tokens)

  return 2 * (prec * rec) / (prec + rec)

In [6]:
def give_an_answer(context,query,answer):

  prediction = predict(context,query)
  em_score = compute_exact_match(prediction, answer)
  f1_score = compute_f1(prediction, answer)

  print(f"Question: {query}")
  print(f"Prediction: {prediction}")
  print(f"True Answer: {answer}")
  print(f"EM: {em_score}")
  print(f"F1: {f1_score}")
  print("\n")

## ***Step 4:*** Test my model

Here, I provide various instances to the model to gauge its proficiency following its training. Initially, I introduced simpler instances before progressively presenting more intricate ones.

In extractive text-based Question Answering tasks, two standard metrics are typically employed to assess performance: one that ascertains an absolute match and another that evaluates the degree of partial alignment.
- **Exact Match**: This metric is concerned with the accuracy of the predicted response in relation to the actual answer. A complete correspondence results in a score of 1.0, while any deviation is scored as 0.0.
- **F1 Score**: This metric calculates the harmonic mean of the overlap in words between the predicted and actual answers. It aims to simultaneously maximize both precision and recall, ensuring a balanced evaluation.

#### As you can see my model predicted all the answers correct in a very small an easy example.

In [12]:
context = "Hi! My name is Alexa and I am 21 years old. I used to live in Peristeri of Athens, but now I moved on in Kaisariani of Athens."

queries = ["How old is Alexa?",
           "Where does Alexa live now?",
           "Where Alexa used to live?"
          ]
answers = ["21",
           "Kaisariani of Athens",
           "Peristeri of Athens"
          ]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)

Question: How old is Alexa?
Prediction: alexa? [SEP] hi! my name is alexa
True Answer: 21
EM: 0
F1: 0


Question: Where does Alexa live now?
Prediction: alexa
True Answer: Kaisariani of Athens
EM: 0
F1: 0


Question: Where Alexa used to live?
Prediction: alexa
True Answer: Peristeri of Athens
EM: 0
F1: 0




In this experiment, I utilized content from Wikipedia pages to assess my model's performance. I noticed that when querying for answers involving multiple entities, particularly those delineated by commas in the context, the model consistently returns only the initial entity (as observed in queries about band members). Furthermore, in inquiring about the band's genre, the model incorrectly supplied "British rock" as an answer, addressing the band's origin instead of the intended question about the type of band.

In [13]:
context = """ Queen are a British rock band formed in London in 1970. Their classic line-up was Freddie Mercury (lead vocals, piano),
            Brian May (guitar, vocals), Roger Taylor (drums, vocals) and John Deacon (bass). Their earliest works were influenced
            by progressive rock, hard rock and heavy metal, but the band gradually ventured into more conventional and radio-friendly
            works by incorporating further styles, such as arena rock and pop rock. """

queries = ["When did Queen found?",
           "Who were the basic members of Queen band?",
           "What kind of band they are?"
          ]
answers = ["1970",
           "Freddie Mercury, Brian May, Roger Taylor and John Deacon",
           "rock"
          ]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)

Question: When did Queen found?
Prediction: band formed
True Answer: 1970
EM: 0
F1: 0


Question: Who were the basic members of Queen band?
Prediction: band formed in london in 1970. their classic line - up was freddie mercury ( lead vocals, piano ), brian may ( guitar, vocals ), roger
True Answer: Freddie Mercury, Brian May, Roger Taylor and John Deacon
EM: 0
F1: 0.33333333333333326


Question: What kind of band they are?
Prediction: band formed
True Answer: rock
EM: 0
F1: 0




#### I noticed that with arithmetic answers and with questions that have the same words as the context, model went very well.

In [14]:
context = """ Mount Olympus is the highest mountain in Greece. It is part of the Olympus massif near
              the Gulf of Thérmai of the Aegean Sea, located in the Olympus Range on the border between
              Thessaly and Macedonia, between the regional units of Pieria and Larissa, about 80 km (50 mi)
              southwest from Thessaloniki. Mount Olympus has 52 peaks and deep gorges. The highest peak,
              Mytikas, meaning "nose", rises to 2917 metres (9,570 ft). It is one of the
              highest peaks in Europe in terms of topographic prominence. """

queries = [
           "How many metres is Olympus?",
           "Where Olympus is near?",
           "How far away is Olympus from Thessaloniki?"
          ]
answers = [
           "2917",
           "Gulf of Thérmai of the Aegean Sea",
           "80 km (50 mi)"
          ]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)

Question: How many metres is Olympus?
Prediction: mount
True Answer: 2917
EM: 0
F1: 0


Question: Where Olympus is near?
Prediction: mount
True Answer: Gulf of Thérmai of the Aegean Sea
EM: 0
F1: 0


Question: How far away is Olympus from Thessaloniki?
Prediction: mount
True Answer: 80 km (50 mi)
EM: 0
F1: 0




Here I give to the model a bigger context and in particular something from COVID-19. At least here it predicted more correct answers than my two first models haha :)

If I asked a question that there isn't any answer in the context or it is irrelevant, then it rightly find no answer.

In [15]:
context = """ The COVID-19 pandemic, also known as the coronavirus pandemic, is an ongoing pandemic of coronavirus disease 2019 (COVID-19)
              caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). It was first identified in December 2019 in Wuhan, China.
              The World Health Organization declared the outbreak a Public Health Emergency of International Concern in January 2020 and a pandemic
              in March 2020. As of 6 February 2021, more than 105 million cases have been confirmed, with more than 2.3 million deaths attributed to COVID-19.
              Symptoms of COVID-19 are highly variable, ranging from none to severe illness. The virus spreads mainly through the air when people are
              near each other.[b] It leaves an infected person as they breathe, cough, sneeze, or speak and enters another person via their mouth, nose, or eyes.
              It may also spread via contaminated surfaces. People remain infectious for up to two weeks, and can spread the virus even if they do not show symptoms.[9]"""

queries = [
           "What is COVID-19?",
           "What is caused by COVID-19?",
           "How many cases have been confirmed from COVID-19?",
           "How many deaths have been confirmed from COVID-19?",
           "How is COVID-19 spread?",
           "How long can an infected person remain infected?",
           "Can a infected person spread the virus even if they don't have symptoms?",
           "What do elephants eat?"
          ]
answers = [
           "an ongoing pandemic of coronavirus disease 2019",
           "severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2)",
           "more than 105 million cases",
           "more than 2.3 million deaths",
           "mainly through the air when people are near each other. It leaves an infected person as they breathe, cough, sneeze, or speak and enters another person via their mouth, nose, or eyes. It may also spread via contaminated surfaces.",
           "up to two weeks",
           "yes",
           ""
          ]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)

Question: What is COVID-19?
Prediction: 
True Answer: an ongoing pandemic of coronavirus disease 2019
EM: 0
F1: 0


Question: What is caused by COVID-19?
Prediction: 
True Answer: severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2)
EM: 0
F1: 0


Question: How many cases have been confirmed from COVID-19?
Prediction: 
True Answer: more than 105 million cases
EM: 0
F1: 0


Question: How many deaths have been confirmed from COVID-19?
Prediction: 
True Answer: more than 2.3 million deaths
EM: 0
F1: 0


Question: How is COVID-19 spread?
Prediction: 
True Answer: mainly through the air when people are near each other. It leaves an infected person as they breathe, cough, sneeze, or speak and enters another person via their mouth, nose, or eyes. It may also spread via contaminated surfaces.
EM: 0
F1: 0


Question: How long can an infected person remain infected?
Prediction: 
True Answer: up to two weeks
EM: 0
F1: 0


Question: Can a infected person spread the virus even if they don't 

Here I gave a context from Harry Potter Wikipedia informations and more difficult questions like "Who is the enemy of Harry Potter?" that didn't find any answer because it doesn't explicitly refer to him as an enemy.

In [18]:
context = """ Harry Potter is a series of seven fantasy novels written by British author, J. K. Rowling. The novels chronicle the lives of a young wizard,
              Harry Potter, and his friends Hermione Granger and Ron Weasley, all of whom are students at Hogwarts School of Witchcraft and Wizardry.
              The main story arc concerns Harry's struggle against Lord Voldemort, a dark wizard who intends to become immortal, overthrow the wizard
              governing body known as the Ministry of Magic and subjugate all wizards and Muggles (non-magical people). Since the release of the first novel,
              Harry Potter and the Philosopher's Stone, on 26 June 1997, the books have found immense popularity, positive reviews, and commercial success worldwide.
              They have attracted a wide adult audience as well as younger readers and are often considered cornerstones of modern young adult literature.[2]
              As of February 2018, the books have sold more than 500 million copies worldwide, making them the best-selling book series in history, and have been translated
              into eighty languages.[3] The last four books consecutively set records as the fastest-selling books in history, with the final installment selling roughly
              eleven million copies in the United States within twenty-four hours of its release.  """

queries = [
           "Who wrote Harry Potter's novels?",
           "Who are Harry Potter's friends?",
           "Who is the enemy of Harry Potter?",
           "What are Muggles?",
           "Which is the name of Harry Poter's first novel?",
           "When did the first novel release?",
           "Who was attracted by Harry Potter novels?",
           "How many languages Harry Potter has been translated into? "
          ]
answers = [
           "J. K. Rowling",
           "Hermione Granger and Ron Weasley",
           "Lord Voldemort",
           "non-magical people",
           "Harry Potter and the Philosopher's Stone",
           "26 June 1997",
           "a wide adult audience as well as younger readers",
           "eighty"
          ]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)

Question: Who wrote Harry Potter's novels?
Prediction: novels written by british author, j. k. rowling. the novels chronicle the lives of a young wizard, harry potter, and his friends hermione granger and ron weasley, all of whom are students at hogwarts school of witchcraft and wizardry. the main story arc concerns harry ' s struggle against lord voldemort, a dark wizard who intends to become immortal, overthrow the wizard governing body known as the ministry of magic and subjugate all wizards and muggles ( non - magical people ). since the release of the first novel, harry potter and the philosopher ' s stone, on 26 june 1997, the books have found immense popularity, positive reviews, and commercial success worldwide. they have attracted a wide adult audience as well as younger readers and are often considered cornerstones of modern young adult literature. [ 2 ] as of february 2018, the books have sold
True Answer: J. K. Rowling
EM: 0
F1: 0.046511627906976744


Question: Who are Harr

## ***Step 5:*** Summaries:
